### Statistics Development Notebook


In [19]:
%load_ext autoreload
%autoreload 2

import glob
import os
from copy import deepcopy


import liftout
import pandas as pd
import plotly.express as px
import streamlit as st
from autoscript_sdb_microscope_client.structures import AdornedImage
from liftout.structures import (AutoLiftoutState, Lamella, Sample,
                                load_experiment)
from liftout.tools.data import (AutoLiftoutStatistics,
                                calculate_statistics_dataframe,
                                create_history_dataframe)
pd.set_option('display.max_rows', None)
EXPERIMENT_PATH = r"C:\Users\Admin\Github\autoliftout\liftout\log\dry-run-01-2022-11-09.01-30-41PM"
stats = calculate_statistics_dataframe(EXPERIMENT_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
display(stats.ml)

,feature,lamella,stage,step
0,NeedleTip,01-proper-dingo,Liftout,NEEDLE_EB_DETECTION
1,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_EB_DETECTION
2,NeedleTip,01-proper-dingo,Liftout,NEEDLE_EB_DETECTION
3,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
4,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
5,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
6,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
7,LamellaRightEdge,01-proper-dingo,Liftout,NEEDLE_SEVER_LAMELLA
8,LamellaRightEdge,01-proper-dingo,Liftout,NEEDLE_SEVER_LAMELLA
9,LamellaRightEdge,01-proper-dingo,Landing,LAND_LAMELLA_ON_POST
